In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, get_train
from manual_nn import nn_pass
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [4]:
data, params = prd.load_caps_train('traindata.mat')

In [3]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven14'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = np.max(params[:,0])
data_load = True
test = True
train = False

n_dof = np.max(params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
mlp_beta = MLPbeta(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler = prd.prep_train_caps(data, params)

# Train neural networks
models = [mlp, mlp_beta, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    train_mod = get_train()

    for epoch in range(sess.epochs):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y in ds:
            train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == sess.epochs-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train aligned LDA
y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

mlp_enc = mlp.get_layer(name='enc')
w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

mlpbeta_enc = mlp_beta.get_layer(name='enc')
w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)

cnn_enc = cnn.get_layer(name='enc')
temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

mlp_w = mlp.get_weights()
mlpb_w = mlp_beta.get_weights()
cnn_w = cnn.get_weights()

Running AB1
no training or testing data
Running AB2
LDA ---- Clean: 90.23, Noisy: 58.54
AUG ---- Clean: 56.60, Noisy: 53.51
MLP ---- Clean: 83.94, Noisy: 75.29, LDA Clean: 85.00, LDA Noisy: 78.09
MLPB ---- Clean: 92.34, Noisy: 86.57, LDA Clean: 92.83, LDA Noisy: 86.80
CNN ---- Clean: 89.09, Noisy: 82.94, LDA Clean: 89.11, LDA Noisy: 82.40
Running AB3
LDA ---- Clean: 76.14, Noisy: 60.57
AUG ---- Clean: 57.60, Noisy: 51.94
MLP ---- Clean: 66.69, Noisy: 65.31, LDA Clean: 66.54, LDA Noisy: 66.14
MLPB ---- Clean: 81.43, Noisy: 79.20, LDA Clean: 80.80, LDA Noisy: 79.63
CNN ---- Clean: 75.00, Noisy: 73.63, LDA Clean: 77.57, LDA Noisy: 75.91
Running AB4
LDA ---- Clean: 86.49, Noisy: 65.66
AUG ---- Clean: 51.40, Noisy: 48.03
MLP ---- Clean: 78.29, Noisy: 75.20, LDA Clean: 78.46, LDA Noisy: 75.23
MLPB ---- Clean: 84.46, Noisy: 81.31, LDA Clean: 84.34, LDA Noisy: 81.86
CNN ---- Clean: 83.71, Noisy: 81.77, LDA Clean: 84.69, LDA Noisy: 82.34
Running AB5
LDA ---- Clean: 94.06, Noisy: 77.06
AUG ---- 